# Assignment

* Load the **houseprices** data from Thinkful's database.
* Reimplement your model from the previous checkpoint.
* Try OLS, Lasso, Ridge, and ElasticNet regression using the same model specification. This time, you need to do **k-fold cross-validation** to choose the best hyperparameter values for your models. Scikit-learn has RidgeCV, LassoCV, and ElasticNetCV that you can utilize to do this. Which model is the best? Why?

## Load Data

In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

# Load data from PostgreSQL database and print out
# observations
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

house_df = pd.read_sql_query('select * from houseprices',con=engine)

# No need for an open connection, as we're only doing a single query
engine.dispose()

house_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [2]:
# Convert street to numerical variable
house_df['street_is_paved'] = np.where(house_df['street'] == 'Pave', 1, 0)

## Reimplement Model

In [3]:
# Y is the target variable
Y = house_df['saleprice']

# X is the feature set
# Replace overallqual_above_6 dummy variable with overallqual
# Add garagecars and mssubclass variables to feature set
X = house_df[['street_is_paved', 'overallqual', 'lotarea', 'totalbsmtsf', 'grlivarea', 'garagearea', 'garagecars', 'mssubclass']]

# Split data into train and test sets using scikit-learn's
# train_test_split() method. The train_test_split() method
# uses the test_size parameter to decide on how much of the
# data will be split out as test data.
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=465)

print("The number of observations in training set is {}".format(X_train.shape[0]))
print("The number of observations in test set is {}\n".format(X_test.shape[0]))

# Add a contant to the model, the 'baseline' of
# the data in case all of your feature values are 0
X_train = sm.add_constant(X_train)

# Fit an OLS model using statsmodels
results = sm.OLS(y_train, X_train).fit()

# Print summary results
print(results.summary())

The number of observations in training set is 1168
The number of observations in test set is 292

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.767
Method:                 Least Squares   F-statistic:                     482.1
Date:                Fri, 03 Jan 2020   Prob (F-statistic):               0.00
Time:                        08:20:32   Log-Likelihood:                -13970.
No. Observations:                1168   AIC:                         2.796e+04
Df Residuals:                    1159   BIC:                         2.800e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

## OLS

In [4]:
# Y is the target variable
Y = house_df['saleprice']

# X is the feature set
X = house_df[['street_is_paved', 'overallqual', 'lotarea', 'totalbsmtsf', 'grlivarea', 'garagearea', 'garagecars', 'mssubclass']]

# Split into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=465)

print("The number of observations in training set is {}".format(X_train.shape[0]))
print("The number of observations in test set is {}\n".format(X_test.shape[0]))

# We fit an OLS model using sklearn
lrm = LinearRegression()
lrm.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lrm.predict(X_train)
y_preds_test = lrm.predict(X_test)

print("R-squared of the model in the training set is: {}\n".format(lrm.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model in the test set is: {}".format(lrm.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

The number of observations in training set is 1168
The number of observations in test set is 292

R-squared of the model in the training set is: 0.7689159464681584

-----Test set statistics-----
R-squared of the model in the test set is: 0.7737012243317923
Mean absolute error of the prediction is: 25110.18741645417
Mean squared error of the prediction is: 1519303013.2929661
Root mean squared error of the prediction is: 38978.23768839436
Mean absolute percentage error of the prediction is: 15.517698697698426


## Lasso

In [5]:
# Create list of alphas to cross-validate against
alphas = np.logspace(-10, 1, 400)

lasso_cv = LassoCV(alphas=alphas, cv=5) 
lasso_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = lasso_cv.predict(X_train)
y_preds_test = lasso_cv.predict(X_test)

print('Best alpha value is: {}\n'.format(lasso_cv.alpha_))
print("R-squared of the model on the training set is: {}\n".format(lasso_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(lasso_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 2.32228109627772

R-squared of the model on the training set is: 0.7689157215957456

-----Test set statistics-----
R-squared of the model on the test set is: 0.7736721475854341
Mean absolute error of the prediction is: 25112.20090262375
Mean squared error of the prediction is: 1519498225.9635978
Root mean squared error of the prediction is: 38980.74173182955
Mean absolute percentage error of the prediction is: 15.521244737885839


## Ridge

In [6]:
ridge_cv = RidgeCV(alphas=alphas, cv=5) 
ridge_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = ridge_cv.predict(X_train)
y_preds_test = ridge_cv.predict(X_test)

print('Best alpha value is: {}\n'.format(ridge_cv.alpha_))
print("R-squared of the model on the training set is: {}\n".format(ridge_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(ridge_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 10.0

R-squared of the model on the training set is: 0.7688423721150601

-----Test set statistics-----
R-squared of the model on the test set is: 0.7735422433452944
Mean absolute error of the prediction is: 25075.389951862744
Mean squared error of the prediction is: 1520370364.5905125
Root mean squared error of the prediction is: 38991.92691558744
Mean absolute percentage error of the prediction is: 15.52133157799739


## ElasticNet

In [7]:
elasticnet_cv = ElasticNetCV(alphas=alphas, cv=5) 
elasticnet_cv.fit(X_train, y_train)

# We are making predictions here
y_preds_train = elasticnet_cv.predict(X_train)
y_preds_test = elasticnet_cv.predict(X_test)

print('Best alpha value is: {}\n'.format(elasticnet_cv.alpha_))
print("R-squared of the model on the training set is: {}\n".format(elasticnet_cv.score(X_train, y_train)))
print("-----Test set statistics-----")
print("R-squared of the model on the test set is: {}".format(elasticnet_cv.score(X_test, y_test)))
print("Mean absolute error of the prediction is: {}".format(mean_absolute_error(y_test, y_preds_test)))
print("Mean squared error of the prediction is: {}".format(mse(y_test, y_preds_test)))
print("Root mean squared error of the prediction is: {}".format(rmse(y_test, y_preds_test)))
print("Mean absolute percentage error of the prediction is: {}".format(np.mean(np.abs((y_test - y_preds_test) / y_test)) * 100))

Best alpha value is: 0.03749167443398878

R-squared of the model on the training set is: 0.768737615957719

-----Test set statistics-----
R-squared of the model on the test set is: 0.7738175074477127
Mean absolute error of the prediction is: 24996.881056968672
Mean squared error of the prediction is: 1518522322.8632843
Root mean squared error of the prediction is: 38968.221961789386
Mean absolute percentage error of the prediction is: 15.4678108880472


The ElasticNet model is the best model because it has nearly the same R-squared value on the training set as the other models, but also has a higher R-squared value on the test set along with lower MAE, MSE, RMSE, and MAPE values.